In [3]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from scipy import stats

UNSW_test1 = pd.read_csv("UNSWNB15_testing1_coursework.csv")
UNSW_test2 = pd.read_csv("UNSWNB15_testing2_coursework_no_label.csv")
UNSW_train1 = pd.read_csv("UNSWNB15_training_coursework.csv")

In [4]:

#Training Data 1
X = UNSW_train1.drop(['label','id'],axis=1)
y = UNSW_train1['label']

X_test = UNSW_test1.drop(['label','id'],axis=1)
y_test = UNSW_test1['label']

X['service'] = UNSW_train1['service'].apply(lambda x: "None" if x == "-" else x)
X_test['service'] = UNSW_test1['service'].apply(lambda x: "None" if x == "-" else x)

# Encoding the categorical data
categorical = ['proto', 'service', 'state']
X_encoded = pd.get_dummies(X[categorical], dtype=int)
X_test_encoded = pd.get_dummies(X_test[categorical], dtype=int)


X_encoded, X_test_encoded = X_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

binary_data = ['is_sm_ips_ports', 'is_ftp_login','ct_ftp_cmd']
X_binary = X[binary_data]
X_test_binary = X_test[binary_data]

# Dropping the categorical data and scaling the numerical data

X = X.drop(columns=categorical + binary_data, axis=1)
X_test = X_test.drop(columns=categorical + binary_data, axis=1)

label = X.columns.values.tolist()

scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)


# Concatenating the scaled data with the encoded data and binary data
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)

X = pd.concat([X, X_encoded.reset_index(drop=True),X_binary.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test, X_test_encoded.reset_index(drop=True),X_test_binary.reset_index(drop=True)], axis=1)


X.head()



,0,1,2,3,4,5,6,7,8,9,...,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,is_sm_ips_ports,is_ftp_login,ct_ftp_cmd
0,-0.212895,-0.122934,-0.205782,-0.042719,-0.115957,1.681699,0.712117,-0.821226,1.154326,-0.261200,...,0,0,0,0,0,1,0,0,0,0
1,-0.021315,0.269448,0.474753,-0.026033,-0.079228,-0.555442,-1.494832,-0.573097,-0.360731,-0.247342,...,0,0,0,0,1,0,0,0,1,1
2,-0.096369,-0.060153,-0.104962,-0.038947,-0.105396,-0.556026,-1.188036,1.334925,-0.360816,-0.255357,...,0,0,0,0,1,0,0,0,0,0
3,-0.212894,-0.122934,-0.205782,-0.043244,-0.115957,0.282989,0.712117,-0.821226,-0.037003,-0.261200,...,0,0,0,0,0,1,0,0,0,0
4,-0.115844,-0.028762,-0.054552,-0.034931,-0.107889,-0.555864,0.712117,1.334925,-0.360739,-0.255584,...,0,0,0,0,1,0,0,0,0,0


In [5]:
#Multilayer Perceptron

class MLP:
    def __init__(self,input_size,hidden_size,output_size):
         
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * np.sqrt(1 / input_size)
        self.weights_output_hidden = np.random.randn(hidden_size, output_size) * np.sqrt(1 / hidden_size)

        self.bias_hidden = np.zeros((1,hidden_size))
        self.bias_output = np.zeros((1,output_size))

    def sigmoid(self,x):
            x = np.clip(x, -500, 500)
            return 1/(1+np.exp(-x))
        
    def forward(self,X):
        self.hidden_sum = np.dot(X,self.weights_input_hidden) + self.bias_hidden
        self.hidden_activation = self.sigmoid(self.hidden_sum)
        self.output_sum = np.dot(self.hidden_activation,self.weights_output_hidden) + self.bias_output
        self.output_activation = self.sigmoid(self.output_sum)
        return self.output_activation           

    def backward(self,X,y,output,learning_rate):
        output = np.clip(output, 1e-7, 1 - 1e-7)
        
        output_error = -(y / output - (1 - y) / (1 - output))
        hidden_error = np.dot(output_error,self.weights_output_hidden.T) * self.hidden_activation * (1 - self.hidden_activation)

        self.weights_output_hidden -= learning_rate * np.dot(self.hidden_activation.T,output_error)
        self.bias_output -= learning_rate * np.sum(output_error)

        self.weights_input_hidden -= learning_rate * np.dot(X.T,hidden_error)
        self.bias_hidden -= learning_rate * np.sum(hidden_error)

    def train(self,X,y,learning_rate):
        output = self.forward(X)
        self.backward(X,y,output,learning_rate)
    
    def predict(self,X):
        probabilities = self.forward(X)
        return (probabilities > 0.5).astype(int) 


In [6]:
X = X.values
y = y.values.reshape(-1,1)

X_test = X_test.values
y_test = y_test.values.reshape(-1,1)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report


# Initializing the classifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=0, activation='logistic', hidden_layer_sizes=(5,), solver='adam', max_iter=200,early_stopping=True)
clf.fit(X, y)
clf_predict = clf.predict(X_test)
# Accuracy factors
print('acc for training data: {:.3f}'.format(clf.score(X, y)))
print('acc for test data: {:.3f}'.format(clf.score(X_test, y_test)))
print('MLP Classification report:\n\n', classification_report(y_test, clf_predict))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc for training data: 0.950
acc for test data: 0.888
MLP Classification report:

               precision    recall  f1-score   support

           0       0.76      0.97      0.85      1308
           1       0.98      0.85      0.91      2692

    accuracy                           0.89      4000
   macro avg       0.87      0.91      0.88      4000
weighted avg       0.91      0.89      0.89      4000



In [ ]:

# Initialize the MLP
input_size = X.shape[1]
hidden_size = 23
output_size = 1
mlp = MLP(input_size, hidden_size, output_size)

# Train the MLP
learning_rate = 0.001
epochs = 1000

for epoch in range(epochs):
    mlp.train(X, y, learning_rate)
    if epoch % 10 == 0:
        predictions = mlp.predict(X_test)
        accuracy = np.mean(predictions == y_test) * 100
        print(f"Epoch {epoch}, Accuracy: {accuracy:.2f}%")

# Make predictions
predictions = mlp.predict(X_test)
print("Final Predictions:", predictions.flatten())



Epoch 0, Accuracy: 67.30%
Epoch 10, Accuracy: 67.27%
Epoch 20, Accuracy: 43.82%
Epoch 30, Accuracy: 43.82%
Epoch 40, Accuracy: 43.82%
Epoch 50, Accuracy: 43.82%
Epoch 60, Accuracy: 43.82%
Epoch 70, Accuracy: 67.30%
Epoch 80, Accuracy: 67.30%
Epoch 90, Accuracy: 67.30%
Epoch 100, Accuracy: 67.30%
